Playing with decision trees

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import datasets
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

In [7]:
digits = datasets.load_digits()

n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1))

X_train, X_test, y_train, y_test = train_test_split(digits.images, digits.target, \
                                                    test_size=.4, random_state=0)
X_train = X_train.reshape(X_train.shape[0],64)
X_test = X_test.reshape(X_test.shape[0],64)

In [8]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [10]:
# oof this is terrible!
expected = y_test
predicted = clf.predict(X_test)
print("Classification report for classifier %s:\n%s\n"
      % (clf, classification_report(expected, predicted, digits=5)))
print("Confusion matrix:\n%s" % confusion_matrix(expected, predicted))

Classification report for classifier DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'):
             precision    recall  f1-score   support

          0    0.89394   0.98333   0.93651        60
          1    0.84058   0.79452   0.81690        73
          2    0.88525   0.76056   0.81818        71
          3    0.80822   0.84286   0.82517        70
          4    0.85714   0.85714   0.85714        63
          5    0.87209   0.84270   0.85714        89
          6    0.93590   0.96053   0.94805        76
          7    0.81333   0.93846   0.87143        65
          8    0.71212   0.60256   0.65278        78
          9    0.76829   0.85135   0.80769        74

avg / total    0.83788   0.83866   0.83636       719


Confusion 

In [11]:
clf.get_params()

{'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_split': 1e-07,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'presort': False,
 'random_state': None,
 'splitter': 'best'}

In [13]:
param_grid = [
  {'class_weight': ["balanced", None], 'criterion': ['gini', 'entropy'], 
   'max_features': ["auto", "sqrt", "log2", None], 'max_depth': [1, 2, 4, 8, 16, 32, 64]},
 ]

In [16]:
scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print '\n'

    clf = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=5,
                       scoring='%s_macro' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print '\n'
    print(clf.best_params_)
    print '\n'
    print("Grid scores on development set:")
    print '\n'
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.5f (+/-%0.05f) for %r"
              % (mean, std * 2, params))
    print '\n'

    print("Detailed classification report:")
    print '\n'
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print '\n'
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred, digits=5))
    print '\n'

# Tuning hyper-parameters for precision




/usr/local/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Best parameters set found on development set:


{'max_features': None, 'criterion': 'entropy', 'max_depth': 32, 'class_weight': None}


Grid scores on development set:


0.04124 (+/-0.00633) for {'max_features': 'auto', 'criterion': 'gini', 'max_depth': 1, 'class_weight': 'balanced'}
0.05509 (+/-0.02527) for {'max_features': 'sqrt', 'criterion': 'gini', 'max_depth': 1, 'class_weight': 'balanced'}
0.04549 (+/-0.02039) for {'max_features': 'log2', 'criterion': 'gini', 'max_depth': 1, 'class_weight': 'balanced'}
0.07584 (+/-0.00399) for {'max_features': None, 'criterion': 'gini', 'max_depth': 1, 'class_weight': 'balanced'}
0.17087 (+/-0.05129) for {'max_features': 'auto', 'criterion': 'gini', 'max_depth': 2, 'class_weight': 'balanced'}
0.17163 (+/-0.07321) for {'max_features': 'sqrt', 'criterion': 'gini', 'max_depth': 2, 'class_weight': 'balanced'}
0.18081 (+/-0.08905) for {'max_features': 'log2', 'criterion': 'gini', 'max_depth': 2, 'class_weight': 'balanced'}
0.21338 (+/-0.03327) for {'